<h1>Sentiment Analysis on Amazon Cells Dataset</h1>

The goal of this project is to do Sentiment Analysis, which is a type of classification, on UCI's "Sentiment Labelled Sentences" Dataset. The dataset consists of 3 different sub-datasets, namely Amazon,Yelp, and IMDB's labelled cells datasets. For a matter of simplification, and the fact that each data has different distributions, I've chosen Amazon's dataset for work.

The steps of the project will be as stated: First, we will preprocess the dataset and ommit punctuations, stop words, extra white spaces,tokenize and convert all letters to lowercase, stem and lemmatize the tokenized words. Then, we will train test split the dataset and vectorize the sentences in each set using sklearn's TF-IDF vectorizer to prepare word embeddings and make the datasets ready for classification. Lastly, we will fit two classification models, namely Multinomial Naive Bayes and Logistic Regression and fit them on the train set, predicting the test set and printing some evaluation metric values for the results. In the end a comparison on the results generated by both models will be made.

<h2>1. Preprocessing</h2>

Before we do anything let's load every package we need. For the purpose of preprocessing we will use the famous NLTK library which is a natural language toolkit in python, suitable for preprocessing text data. 

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix , accuracy_score , f1_score
import warnings
warnings.filterwarnings('ignore')

Let's load the amazon dataset in a pandas dataframe, and name the feature column 'Sentence', while naming the class labels column 'Class'. The class label is binary, with value 0 as a negative sentiment obtained from it's corresponding sentence(a negative comment made by an amazon user on a product), and 1 is a positive sentiment(meaning that the comment made by the user has been positive). We will also create a helper dataframe with columns index,class,and sentence, and name it temp. We will use that later as an empty dataframe which helps store some preprocessed values so we can access it later.   

In [2]:
amazon = pd.read_csv('amazon_cells_labelled.txt',delimiter='\t',header=None)
amazon.columns = ['Sentence','Class']
amazon['index'] = amazon.index

columns = ['index','Class', 'Sentence']
temp = pd.DataFrame(columns=columns)
print(amazon.head())

                                            Sentence  Class  index
0  So there is no way for me to plug it in here i...      0      0
1                        Good case, Excellent value.      1      1
2                             Great for the jawbone.      1      2
3  Tied to charger for conversations lasting more...      0      3
4                                  The mic is great.      1      4


Now it's time for some preprocessing. For omitting punctuations, white spaces, and converting letters to lowercase, the pandas dataframe has built in commands, so we won't need nltk for now.

The first step is to omit punctuations, such as .,!,?,...
That will be done with pandas str.replace() which deletes them.

In [3]:
amazon['Sentence'] = amazon['Sentence'].str.replace('[^\w\s]','')
print(amazon.head())

                                            Sentence  Class  index
0  So there is no way for me to plug it in here i...      0      0
1                          Good case Excellent value      1      1
2                              Great for the jawbone      1      2
3  Tied to charger for conversations lasting more...      0      3
4                                   The mic is great      1      4


Now it's time to convert all letters to lowercase, which is done by pandas str.lower() that replaces all uppercase letters with lowercase.

In [4]:
amazon['Sentence'] = amazon['Sentence'].str.lower()
print(amazon.head())

                                            Sentence  Class  index
0  so there is no way for me to plug it in here i...      0      0
1                          good case excellent value      1      1
2                              great for the jawbone      1      2
3  tied to charger for conversations lasting more...      0      3
4                                   the mic is great      1      4


And to omit extra white spaces pandas str.strip() is used. Of course it's not visible below but if the whole dataset is manually checked the result will be viewed.

In [5]:
amazon['Sentence'] = amazon['Sentence'].str.strip()
print(amazon.head())

                                            Sentence  Class  index
0  so there is no way for me to plug it in here i...      0      0
1                          good case excellent value      1      1
2                              great for the jawbone      1      2
3  tied to charger for conversations lasting more...      0      3
4                                   the mic is great      1      4


For the next step, we need to omit some stop words from our dataset, such as in,from,i,etc. That's where we need the nltk library, precisely the corpus and tokenize modules. In these modules exist two tools we need, stopwords and word_tokenize.The steps we'll take will be as followed: We will iterate over each row(sentence)in the pandas dataframe using the iterrows() command, then for each row(sentence)we will tokenize the words using the word_tokenize() tool mentioned above and store the tokens in word_tokens variable. Then, for every word token we will check if it is a stop word using the stopwords.word('english') command which has a database of words defined as stopwords in english built-in. If our word tokens are indeed stopwords,we will omit them and store the filtered sentences(rows) in the filtered_sentence variable. Then, using the empty temp dataframe mentioned at the beginning of the project, we will append the filtered rows in the Sentence column, and their corresponding class label and index in the columns Index and Class. This will be our final dataset, so we will update the amazon dataset with this new version with the help of the temp dataframe. We then will empty the temp dataframe again as we need it for the next steps.    

In [6]:
for index, row in amazon.iterrows():
    word_tokens = word_tokenize(row['Sentence'])
    filtered_sentence = [w for w in word_tokens if not w in stopwords.words('english')]
    temp = temp.append({"index": row['index'], "Class":  row['Class'],"Sentence": " ".join(filtered_sentence[0:])}, ignore_index=True)
amazon = temp
temp = pd.DataFrame(columns=columns)
print(amazon.head())

  index Class                                           Sentence
0     0     0                    way plug us unless go converter
1     1     1                          good case excellent value
2     2     1                                      great jawbone
3     3     0  tied charger conversations lasting 45 minutesm...
4     4     1                                          mic great


Now it's time for stemming and lemmatization. First, a brief on stemming.

Stemming is a normalization technique used for sets of words in sentences which converts them into a sequence for shorter lookup process. The words which have similar meanings with some variations in context are chosen for the task. In other words, all the chosen words have one root word, but all of them are different variations of that root word. As an example, if the root word is 'eat', it's variations are 'eats','eaten','eating',etc. Stemming is used to find that root word vor any variations of words. It does so because as simple as it might seem to us, the machines don't identify the similarity in meaning for different variations of the same root word. As a result classification and prediction is not done correctly. To escape those problems stemming categorizes the same type of data(words) by getting their root word.

Now, back to our work. We will use the PorterStemmer module from nltk.stem for the stemming task. Similar to the method used above for the stop words, we will iterate each sentence in the dataframe, tokenize each word, then for every word token we will stem it using stem() and append the stemmed sentences and their corresponding index and class label to temp. Lastly, we will update the amazon dataset and clear the temp dataframe. 

In [7]:
ps = PorterStemmer()

for index, row in amazon.iterrows():
    word_tokens = word_tokenize(row['Sentence'])
    stemmed_sentence = [ps.stem(w) for w in word_tokens]
    temp = temp.append({"index": row['index'], "Class":  row['Class'],"Sentence": " ".join(stemmed_sentence[0:])}, ignore_index=True)
amazon = temp
temp = pd.DataFrame(columns=columns)
print(amazon.head())

  index Class                                          Sentence
0     0     0                     way plug us unless go convert
1     1     1                              good case excel valu
2     2     1                                      great jawbon
3     3     0  tie charger convers last 45 minutesmajor problem
4     4     1                                         mic great


The last part of our preprocessing is dedicated to lemmatization.

Lemmatization is also a technique of converting a word to its base(root). The main difference between stemming and lemmatization is that lemmatization focuses on the context of the word and converts them to a meaningful root word, whereas stemming simply removes the last few letters of the words and does not pay any attention to the context. That often leads to some spelling errors in the words.

Combined together, stemming and lemmatization are useful preprocessing steps. 

We will use the WordNetLemmatizer module from nltk.stem which is a large lexical database offered by nltk. There are other versions available,such as Spacy, TextBlob, CLiPS pattern, TreeTagger, Gensim Lemmatizer,... but we won't get into them. 
The process for lemmatizing is exactly the same as with stemming, with the difference that we use lemmatize() for each word token. So I won't write the steps again.

In [8]:
wl = WordNetLemmatizer()

for index, row in amazon.iterrows():
    word_tokens = word_tokenize(row['Sentence'])
    lemmatized_sentence = [wl.lemmatize(w) for w in word_tokens]
    temp = temp.append({"index": row['index'], "Class":  row['Class'],"Sentence": " ".join(lemmatized_sentence[0:])}, ignore_index=True)
amazon = temp
print(amazon.head())

  index Class                                          Sentence
0     0     0                      way plug u unless go convert
1     1     1                              good case excel valu
2     2     1                                      great jawbon
3     3     0  tie charger convers last 45 minutesmajor problem
4     4     1                                         mic great


<h2>2. Classification</h2>

Now that the main preprocessing steps are done, it's time for some additional steps before the classification. First we'll split the amazon dataset into variables x and y to store the sentences and class labels, respectively. At the same time we will convert the sentences dataframe into a unicode numpy array and the labels dataframe into an int32 numpy array for better classification. Then we'll train test split the data into training and test sets with a test size of 0.2.

In [9]:
x = amazon['Sentence'].values.astype('U')
y = amazon['Class'].values.astype('int32')
print(type(x))
print(type(y))

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42 )

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Now before the classification, we need a word embedding for the raw text sentences we have. If we don't do that, the classification won't be executed correctly because it doesn't recognize raw text. There are many embedding techniques available such as ElMO,Google Bert,Count Vectorizer, TF-IDF vectorizer, NGram and CharLevel Vectorizers,etc. For this project we'll use TF-IDF. 

TF-IDF is an abbreviation for Term Frequency-Inverse Document Frequency and is a common vectorizer used to transform raw text into meaningful number representations. It converts sentences to a matrix of tf-idf features. Vectorization is generally the process of converting text data into numerical feature vectors that describe the word occurrences(number of times a specific word was seen in the text) while ignoring the position of the words in the raw text data. Count Vectorizer tokenizes words and counts the number of occurrences of words in a single class.TF-IDF does a little more. In large text datasets, some words are too frequent(like is,a,...)but have little useful information about the actual context of the text. TF-IDF re-weights the feature vectors by decreasing the effect of such words mentioned above. I won't get into the math of what it does, but the complete documentation is available in sklearn: https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

We will vectorize the train and test sets and store the results in X_train_v and X_test_v,which will then be used for classification.

In [10]:
vectorizer = TfidfVectorizer(analyzer='word') 
X_train_v = vectorizer.fit_transform(X_train)
X_test_v = vectorizer.transform(X_test)

We will use two classification models, Naive Bayes and Logistic Regression. For Naive Bayes, we will use sklearn's MultinomialNB(), which(according to sklearn's documentation) is a multinomial Naive Bayes suitable for classification with discrete features such as word counts or tf-idf features in text classification. We will fit the model on the train set(the new vectorized one) and predict the model using the test set.  

In [11]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_v, y_train)
y_pred = naive_bayes.predict(X_test_v)

For the second part, we will build a LogisticRegression() model from sklearn and fit it on the vectorized train set,while predicting the test set. 

In [12]:
Logistic_Regression = LogisticRegression()
Logistic_Regression.fit(X_train_v, y_train)
y_pred_2 = Logistic_Regression.predict(X_test_v)

<h3>3. Evaluation & Comparison</h3>

For the labels predicted by the two models, we will print their confusion matrices, accuracies, errors, and F1-Scores and compare the results. Here are the metrics for the Naive Bayes classifer:

In [13]:
print('Test Confusion Matrix:')
print(confusion_matrix(y_test,y_pred))
print('Test F1-Score: ', f1_score(y_test,y_pred))
print('Test Accuracy: ', accuracy_score(y_test,y_pred))
print('Test Error: ', 1-accuracy_score(y_test,y_pred))

Test Confusion Matrix:
[[75 18]
 [17 90]]
Test F1-Score:  0.8372093023255813
Test Accuracy:  0.825
Test Error:  0.17500000000000004


And here are the results obtained from Logistic Regression:

In [14]:
print('Test Confusion Matrix:')
print(confusion_matrix(y_test,y_pred_2))
print('Test F1-Score: ', f1_score(y_test,y_pred_2))
print('Test Accuracy: ', accuracy_score(y_test,y_pred_2))
print('Test Error: ', 1-accuracy_score(y_test,y_pred_2))

Test Confusion Matrix:
[[78 15]
 [25 82]]
Test F1-Score:  0.8039215686274509
Test Accuracy:  0.8
Test Error:  0.19999999999999996


Let's compare the results. Naive Bayes has higher accuracy and F1-score and less error compared to Logistic Regression(0.83-0.82-0.17 vs 0.80-0.8-0.19). According to their confusion matrices, Naive bayes has classified 165 labels correctly and 35 labels incorrectly, while Logistic Regression has correctly classified 160 labels and misclassied 40 labels. So overall, Naive Bayes has had a better performance for our data. The dataset is not imbalanced according to UCI, and we have nearly similar quantities of 0 and 1 labels, so the f1-score and accuracy score are close to each other and it doesn't make a crucial difference to consider any of them for the evaluation metric. By doing more preprocessing or using other state of the art vectorizing method the accuracies and f1-scores might increase and the models might give better results.  